# Super-agent boost
Create a new strategy to compete in a multi-agent environment. For this, the bid prices will be boosted for the selected pCTR estimator and bidding strategy combination, according to the calculated pCTRs

In [1]:
#Import libraries
import os
import random
import numpy as np
import pandas as pd
import math
from tqdm import tnrange, tqdm_notebook


from glob import glob
from IPython.display import display

In [2]:
%%time

#Load pCTRs
Path = 'C://Datasets//Multiagent//calc//pCTRs'

os.chdir(Path)
Validation_CTR = {}
Test_CTR = {}

for Filename in glob('*validation_pCTR.csv'):
    Validation_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')
    
for Filename in glob('*test_pCTR.csv'):
    Test_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')
    
#Load bids
Path = 'C://Datasets//Multiagent//calc//bids'
os.chdir(Path)

Bids = {}
for Filename in glob('*bids.csv'):
    #print (Filename[8:-9])
    Bids[Filename[8:-9]] = pd.read_csv(Filename, sep = ',')


Wall time: 13.4 s


#### Define super-agent
Select bidding strategy/ pCTR estimator algorithm

In [3]:
# Define super-agent strategy
best_single_algorithm = 'NN'
best_single_bidding_strategy = 'Ortb'
best_params_bid_strategy = []

get_bids = Bids[best_single_algorithm+'_'+best_single_bidding_strategy]
get_bids.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,317.4
1,29167d4caa719788b5a342dbaa25151d53121f80,432.4
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,30.4
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,130.2
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,77.7


#### Create superagent dataframe
Combine pCTRs and original bid prices into single dataframe

In [4]:
superagent_df = pd.concat([
    Test_CTR[best_single_algorithm+'_test_pCTR']['bidid'],
    Test_CTR[best_single_algorithm+'_test_pCTR']['pCTR'],
    Bids[best_single_algorithm+'_'+best_single_bidding_strategy]['bidprice']
], axis=1)
 
superagent_df.rename(columns={"bidprice": "originalbid"},inplace=True)
superagent_df.sort_values('pCTR', ascending=False).head(10)

,bidid,pCTR,originalbid
121051,0246d64b49fa8269ce10e4f46b767e6968d8b381,0.286297,7181.5
172343,01c0c1a83de1c8bd3cab8299a8e4d55ef19a57d2,0.284037,7152.5
202026,91c2b30a1642bd86c3d9cbd947cd6de514e87a33,0.275505,7042.1
174109,1ce75f7dcf499a312b851780fd62e3a6cd8f1be4,0.263722,6886.8
196414,7e169d7b8d15a594ffc73aaed4757c1640c86a50,0.245325,6637.3
190433,67252830e07cedbad0a9d5d8d2ef2896b44837a1,0.245321,6637.2
224409,d050d7edff07a5b662957044c4b1bd70616d452e,0.236658,6516.5
41650,0e8f668713bbd44fd8cfabdb74ac790da0bfff06,0.220355,6283.0
232926,06624b0bfc6e830b84451f38802ec8d880c926d7,0.220118,6279.6
216941,0023961d140331981212034fd128b0c388603b4a,0.217830,6246.1


#### Boosted bid creations
Cluster pCTRs into bins, create factor, create new bid prices based on binFactor * originalBidPrice

In [5]:
numBins = 20
factor = np.linspace(1.05,3.5,numBins)
superagent_df['factor'] = pd.cut(superagent_df['pCTR'],numBins, labels=factor)
superagent_df['bidprice'] = (superagent_df['originalbid'] * np.single(superagent_df['factor']))
superagent_df.bidprice = superagent_df.bidprice.round(1)
max_bid = 6000
superagent_df.loc[superagent_df['bidprice'] > max_bid,'bidprice'] = max_bid #Define max bids
display(superagent_df.sort_values('originalbid', ascending=False).head())


,bidid,pCTR,originalbid,factor,bidprice
121051,0246d64b49fa8269ce10e4f46b767e6968d8b381,0.286297,7181.5,3.500000,6000.0
172343,01c0c1a83de1c8bd3cab8299a8e4d55ef19a57d2,0.284037,7152.5,3.500000,6000.0
202026,91c2b30a1642bd86c3d9cbd947cd6de514e87a33,0.275505,7042.1,3.500000,6000.0
174109,1ce75f7dcf499a312b851780fd62e3a6cd8f1be4,0.263722,6886.8,3.371053,6000.0
196414,7e169d7b8d15a594ffc73aaed4757c1640c86a50,0.245325,6637.3,3.242105,6000.0


#### Write new bids

In [6]:
superagent_df.to_csv(Path + '//Team_07_Superagent.csv',columns=['bidid','bidprice'], index=False)
print('Completed')

Completed


In [7]:
#Future work: try this instead of linearly spaced increases for factor

#np.logspace(0.0,1,10)/10